In [ ]:
# pip install opencv-python opencv-python-headless scikit-image

Программа нахождения различающихся кадров

In [ ]:
import os
import cv2
from skimage.metrics import structural_similarity as ssim
from google.colab import drive   # Подключение диска
drive.mount('/content/drive/')   # Подключить диск


In [ ]:
# Программа проходит по всем файлам в указанной папке, сравнивает каждый кадр с предыдущим и сохраняет только те кадры,
# которые имеют достаточно низкую степень сходства с предыдущим кадром, определенную пороговым значением similarity_threshold
# Arc Аркадий Соловьев

input_folder = '/content/drive/MyDrive/Colab Notebooks/Видео/NEW_zoomcam2'  # Заменить это на путь к нужной папке
output_folder = '/content/drive/MyDrive/Непохожие файлы'                    # Папка для нового отфильтрованного датасета
os.makedirs(output_folder, exist_ok=True)                                   # Создать выходную папку

def image_similarity(image1, image2):
    # Функция вычисляет структурный коэффициент схожести (SSI) между изображениями

    return ssim(image1, image2, multichannel=True)  # Функция возвращает структурный коэффициент схожести

def filter_duplicate_frames(input_folder, output_folder, similarity_threshold=0.9):
    # Функция принимает 3 параметра; 1 - input_folder - папка с входными кадрами;
    # 2 - output_folder - папка для сохранения отфильтрованных кадров;
    # 3 - необязательный параметр, определяющий минимальное значение сходства для того, чтобы считать два кадра дубликатами.
    # По умолчанию, это значение равно 0.9, в программе его можно заменить.
    previous_frame = None # Эта строка инициализирует переменную previous_frame как None.
                          # В эту переменную будет сохраняться предыдущий кадр для сравнения с текущим.

    # Цикл перебирает все файлы в указанной input_folder (входной папке)
    for filename in os.listdir(input_folder):
        if filename.endswith(".png"):  # Эта строка проверяет, является ли файл PNG.
                                       # Изменить формат файла, если он отличается от PNG
            # Эта строка загружает текущий кадр изображения, используя OpenCV функцию imread,
            # которая принимает путь к файлу и возвращает изображение в виде массива пикселей.
            current_frame = cv2.imread(os.path.join(input_folder, filename))

            # Если previous_frame равно None, значит это первый кадр в цикле.
            if previous_frame is None:
                previous_frame = current_frame # Сохраняем текущий кадр в переменную previous_frame
                output_filename = os.path.join(output_folder, filename) # Создаем путь для сохранения кадра в выходную папку
                cv2.imwrite(output_filename, current_frame) # Сохраняем текущий кадр в указанной выходной папке
            else:
                # Вычисляем степень сходства между предыдущим и текущим кадрами. Результат сохраняется в переменной similarity.
                similarity = image_similarity(previous_frame, current_frame)

                if similarity < similarity_threshold: # Если коэффициент схожести меньше порогового значения
                    # Создаем путь для сохранения текущего кадра в выходную папку
                    output_filename = os.path.join(output_folder, filename)
                    cv2.imwrite(output_filename, current_frame) # Сохраняем текущий кадр в указанной выходной папке
                    previous_frame = current_frame # Обновляем предыдущий кадр на текущий для следующей итерации цикла

similarity_threshold = 0.9  # Настроить коэффициент схожести по своему усмотрению

# Фильтрация похожих файлов
filter_duplicate_frames(input_folder, output_folder, similarity_threshold)


<ipython-input-31-09604ef5c1a7>:10: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  return ssim(image1, image2, multichannel=True)  # similarity = ssim(image1, image2, win_size=(5, 5))  1080, 1920


Метрика SSI (Structural Similarity Index) используется для измерения структурного сходства между двумя изображениями. Она предоставляет числовую оценку, насколько изображения похожи друг на друга с точки зрения их структуры. SSI считается более точным и информативным в сравнении с простым сравнением пикселей или среднеквадратичным отклонением, так как учитывает восприятие человека и структурные особенности изображений.

SSI вычисляется на основе трех ключевых компонентов:

Яркость (L): Изображения сначала преобразуются в черно-белый формат (оттенки серого), чтобы учесть только информацию о яркости пикселей. Это делается потому, что структурное сходство не зависит от цвета.

Контраст (C): Оценка различия в контрасте между двумя изображениями. Она измеряется как стандартное отклонение значений пикселей.

Структура (S): Эта часть SSI измеряет, насколько структура изображения (текстуры, детали и т. д.) воспринимается человеческим зрением. Для этого используется метод сравнения блоков изображения, где каждое изображение разбивается на множество небольших блоков, и сравниваются соответствующие блоки между изображениями.

SSI вычисляется как комбинация этих компонентов и предоставляет значение в диапазоне от -1 до 1, где:

Значение 1 указывает на полное сходство между изображениями.
Значение 0 указывает на полное отсутствие сходства.
Значение -1 указывает на полную противоположность.